# Lezione 20 — K-Means Clustering

---

## Obiettivi della Lezione

Al termine di questa lezione sarai in grado di:

1. **Comprendere** la geometria del clustering K-Means
2. **Spiegare** il ruolo dei centroidi e come vengono aggiornati
3. **Calcolare** la distanza euclidea e capire perché è fondamentale
4. **Riconoscere** le assunzioni forti del modello e i loro limiti
5. **Applicare** K-Means correttamente con sklearn

---

## Perché questa lezione è importante

K-Means è l'algoritmo di clustering più usato al mondo. È semplice, veloce e spesso efficace.

Ma questa semplicità nasconde **assunzioni forti** che, se ignorate, portano a risultati sbagliati:
- Assume che i cluster siano **sferici**
- Assume che abbiano **dimensioni simili**
- È **sensibile** ai valori iniziali e agli outlier

Capire come funziona *davvero* K-Means ti permette di:
- Usarlo quando è appropriato
- Evitarlo quando non lo è
- Interpretare correttamente i risultati

---

## Ruolo nel percorso

| Lezione | Argomento |
|---------|-----------|
| 19 | Introduzione all'Unsupervised Learning |
| **20** | **K-Means Clustering** ← Sei qui |
| 21 | Scelta del numero di cluster (Elbow, Silhouette) |
| 22 | Clustering Gerarchico |
| 23 | DBSCAN |

Questa lezione introduce il **primo algoritmo concreto** di clustering. Nelle lezioni successive vedremo come scegliere K e algoritmi alternativi.

---

# Parte 1 — Teoria Concettuale

---

## 1.1 Cos'è il clustering?

Il **clustering** è il task di raggruppare osservazioni simili tra loro, senza sapere a priori quanti gruppi esistono o quali sono.

### L'idea intuitiva

Immagina di avere 1000 clienti descritti da:
- Frequenza di acquisto
- Spesa media
- Tempo dall'ultimo acquisto

Senza sapere nulla di loro, vuoi trovare **gruppi naturali**: clienti che si comportano in modo simile.

Il clustering risponde alla domanda:
> "Quali osservazioni stanno insieme?"

---

## 1.2 K-Means: l'idea fondamentale

K-Means è l'algoritmo di clustering più semplice e diffuso.

### Il concetto in una frase

> K-Means divide i dati in **K gruppi** minimizzando la distanza tra ogni punto e il **centro del suo gruppo** (centroide).

### Cosa sono i centroidi?

Un **centroide** è il punto medio di un cluster: la media aritmetica di tutte le osservazioni che appartengono a quel gruppo.

$$\mu_k = \frac{1}{|C_k|} \sum_{x_i \in C_k} x_i$$

Dove:
- $\mu_k$ = centroide del cluster $k$
- $C_k$ = insieme dei punti assegnati al cluster $k$
- $|C_k|$ = numero di punti nel cluster $k$

**Esempio numerico:**

Se un cluster contiene 3 punti in 2D:
- Punto A: (2, 4)
- Punto B: (4, 6)
- Punto C: (3, 5)

Il centroide è:
$$\mu = \left(\frac{2+4+3}{3}, \frac{4+6+5}{3}\right) = (3, 5)$$

---

## 1.3 La distanza euclidea

K-Means usa la **distanza euclidea** per misurare quanto un punto è "lontano" da un centroide.

### Formula in 2 dimensioni

Per due punti $A = (x_1, y_1)$ e $B = (x_2, y_2)$:

$$d(A, B) = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}$$

È il teorema di Pitagora: la distanza in linea retta.

### Formula generale (n dimensioni)

Per due punti con $n$ feature:

$$d(A, B) = \sqrt{\sum_{i=1}^{n} (a_i - b_i)^2}$$

### Esempio numerico

Punto A: (2, 3, 1)  
Punto B: (5, 7, 2)

$$d(A, B) = \sqrt{(5-2)^2 + (7-3)^2 + (2-1)^2} = \sqrt{9 + 16 + 1} = \sqrt{26} \approx 5.1$$

---

### Perché la distanza euclidea è importante?

K-Means assegna ogni punto al centroide **più vicino** in termini di distanza euclidea.

Questo ha conseguenze importanti:
1. **Scale diverse → problemi**: se una feature ha range 0-100 e un'altra 0-100.000, la seconda domina
2. **Forma sferica implicita**: la distanza euclidea definisce "sfere" attorno ai centroidi
3. **Outlier problematici**: punti molto lontani distorcono i centroidi

---

## 1.4 L'algoritmo K-Means passo per passo

L'algoritmo è iterativo e segue questi passi:

### Passo 0: Inizializzazione
Scegli K punti iniziali come centroidi (random o con metodo k-means++).

### Passo 1: Assegnazione
Per ogni punto, calcola la distanza da tutti i K centroidi.
Assegna il punto al centroide più vicino.

### Passo 2: Aggiornamento
Ricalcola ogni centroide come la media dei punti assegnati a quel cluster.

### Passo 3: Convergenza
Se i centroidi non cambiano (o cambiano meno di una soglia), STOP.
Altrimenti, torna al Passo 1.

---

### Visualizzazione dell'algoritmo

```
Iterazione 0:  ●  ●  ●     (centroidi iniziali random)
               ·  ·  ·  ·  ·  (punti da clusterizzare)

Iterazione 1:  Assegna ogni punto al centroide più vicino
               Ricalcola i centroidi

Iterazione 2:  Riassegna, ricalcola...

...

Convergenza:   I centroidi non si muovono più → FINE
```

---

### La funzione obiettivo (Inertia)

K-Means minimizza l'**inertia** (o Within-Cluster Sum of Squares, WCSS):

$$\text{Inertia} = \sum_{k=1}^{K} \sum_{x_i \in C_k} ||x_i - \mu_k||^2$$

In parole: la somma delle distanze al quadrato di ogni punto dal suo centroide.

**Meno inertia = cluster più compatti.**

Attenzione: l'inertia diminuisce SEMPRE all'aumentare di K (caso limite: K = n punti → inertia = 0).

---

## 1.5 Le assunzioni forti di K-Means

K-Means funziona bene **solo se** i dati rispettano certe condizioni. Se queste assunzioni sono violate, i risultati saranno sbagliati.

---

### Assunzione 1: Cluster sferici (isotropici)

K-Means assume che i cluster abbiano **forma sferica** (o iper-sferica in n dimensioni).

Questo perché usa la distanza euclidea, che definisce "cerchi" di equidistanza attorno ai centroidi.

**Problema:** Se i cluster hanno forma allungata, a "banana", o irregolare, K-Means li taglia male.

---

### Assunzione 2: Cluster di dimensioni simili

K-Means tende ad assegnare lo stesso numero di punti a ogni cluster.

**Problema:** Se un cluster ha 1000 punti e un altro ne ha 50, K-Means potrebbe "rubare" punti dal cluster grande per bilanciare.

---

### Assunzione 3: Varianza simile tra cluster

K-Means assume che i cluster abbiano **dispersione simile** attorno al centroide.

**Problema:** Se un cluster è molto compatto e un altro molto disperso, il confine sarà sbagliato.

---

### Assunzione 4: Assenza di outlier significativi

I centroidi sono **medie**, quindi molto sensibili agli outlier.

**Problema:** Un singolo punto anomalo può spostare significativamente un centroide.

---

### Tabella riassuntiva

| Assunzione | Cosa assume K-Means | Cosa succede se violata |
|------------|---------------------|-------------------------|
| Forma | Cluster sferici | Cluster tagliati male |
| Dimensione | Cluster bilanciati | Punti assegnati al cluster sbagliato |
| Varianza | Dispersione simile | Confini distorti |
| Outlier | Nessun outlier | Centroidi distorti |

---

## 1.6 Il problema dell'inizializzazione

K-Means è un algoritmo **greedy**: trova un minimo locale, non globale.

Il risultato dipende da **dove partono i centroidi iniziali**.

### Il problema

```
Inizializzazione A:  ●    ●    ●     → Converge a soluzione X
Inizializzazione B:  ●  ●      ●     → Converge a soluzione Y (diversa!)
```

Con centroidi iniziali diversi, K-Means può convergere a soluzioni diverse.

### La soluzione: k-means++

L'algoritmo **k-means++** (default in sklearn) sceglie i centroidi iniziali in modo intelligente:

1. Scegli il primo centroide random
2. Per i successivi, scegli punti **lontani** dai centroidi già scelti
3. La probabilità di scegliere un punto è proporzionale alla sua distanza dal centroide più vicino

Questo riduce la probabilità di inizializzazioni sfortunate.

### n_init: eseguire più volte

Sklearn esegue K-Means **n_init volte** con inizializzazioni diverse e tiene la soluzione con inertia minore.

```python
KMeans(n_clusters=3, n_init=10, random_state=42)
```

Significa: esegui 10 volte, tieni il risultato migliore.

---

# Parte 2 — Schema Mentale e Mappa Logica

---

## 2.1 Quando usare K-Means

### Situazioni ideali per K-Means

| Condizione | Perché K-Means funziona |
|------------|-------------------------|
| **Cluster sferici** | La distanza euclidea li cattura bene |
| **Cluster bilanciati** | Nessun cluster viene "schiacciato" |
| **Varianza simile** | I confini saranno corretti |
| **Dati scalati** | Tutte le feature contribuiscono equamente |
| **K noto o stimabile** | Puoi usare Elbow/Silhouette (Lezione 21) |
| **Molti dati** | K-Means è veloce anche su milioni di punti |

### Segnali che K-Means è appropriato

- Scatter plot 2D mostra gruppi "rotondi" e separati
- Le feature hanno scale simili (o le hai scalate)
- Non ci sono outlier evidenti
- Il business suggerisce un numero ragionevole di segmenti

---

## 2.2 Quando NON usare K-Means

### Situazioni problematiche

| Condizione | Perché K-Means fallisce | Alternativa |
|------------|-------------------------|-------------|
| **Cluster allungati** | Li taglia male | DBSCAN, Spectral |
| **Cluster di dimensioni diverse** | Ruba punti | DBSCAN, GMM |
| **Outlier** | Distorcono i centroidi | DBSCAN, rimuovi outlier |
| **K ignoto** | Risultato arbitrario | Hierarchical per esplorare |
| **Forma arbitraria** | Assume sfericità | DBSCAN |

### Segnali che K-Means è inappropriato

- Scatter plot mostra forme "a banana" o irregolari
- Un gruppo è molto più grande degli altri
- Ci sono punti isolati lontani da tutto
- Non hai idea di quanti cluster cercare

---

## 2.3 Checklist pre-K-Means

Prima di applicare K-Means:

- [ ] **Scaling**: hai applicato StandardScaler?
- [ ] **Outlier**: hai verificato e gestito i punti anomali?
- [ ] **K**: hai un'ipotesi su quanti cluster cercare?
- [ ] **Forma**: i dati sembrano avere gruppi "rotondi"?
- [ ] **n_init**: stai usando n_init ≥ 10?
- [ ] **random_state**: hai fissato il seed per riproducibilità?

---

# Parte 3 — Notebook Dimostrativo

---

## Demo 1: K-Means base — vedere l'algoritmo in azione

Generiamo dati con 3 gruppi ben separati e vediamo come K-Means li trova.

In [ ]:
# ============================================
# DEMO 1: K-Means base
# ============================================
# Obiettivo: vedere K-Means in azione su dati ideali

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Fissiamo il seed per riproducibilità
np.random.seed(42)

# ============================================
# PASSO 1: Generazione dati sintetici
# ============================================
# Creiamo 3 cluster ben separati (caso ideale per K-Means)
X, y_true = make_blobs(
    n_samples=300,       # 300 punti totali
    centers=3,           # 3 centri
    cluster_std=0.8,     # deviazione standard di ogni cluster
    random_state=42
)

print("="*60)
print("DEMO 1: K-Means su dati ideali")
print("="*60)
print(f"\nDimensioni dataset: {X.shape}")
print(f"Gruppi veri: {np.unique(y_true)}")

# ============================================
# PASSO 2: Applicazione K-Means
# ============================================
# Nota: in questo caso i dati sono già centrati, ma per buona pratica scaliamo
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Applichiamo K-Means con K=3 (sappiamo che ci sono 3 gruppi)
kmeans = KMeans(
    n_clusters=3,        # numero di cluster
    init='k-means++',    # inizializzazione intelligente
    n_init=10,           # esegui 10 volte, tieni il migliore
    max_iter=300,        # massimo iterazioni per convergenza
    random_state=42      # riproducibilità
)

# fit_predict: addestra e restituisce le etichette
labels = kmeans.fit_predict(X_scaled)

# ============================================
# PASSO 3: Analisi del risultato
# ============================================
print(f"\nRisultato K-Means:")
print(f"  Etichette uniche: {np.unique(labels)}")
print(f"  Punti per cluster: {np.bincount(labels)}")
print(f"  Inertia (WCSS): {kmeans.inertia_:.2f}")
print(f"  Iterazioni per convergere: {kmeans.n_iter_}")

# Silhouette score (metrica di qualità)
sil = silhouette_score(X_scaled, labels)
print(f"  Silhouette score: {sil:.3f}")

# ============================================
# PASSO 4: Visualizzazione
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Gruppi veri (se li conoscessimo)
axes[0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=50, alpha=0.7)
axes[0].set_title('Gruppi VERI (ground truth)')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
axes[0].grid(True, alpha=0.3)

# Plot 2: Clustering K-Means
scatter = axes[1].scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=50, alpha=0.7)

# Trasformiamo i centroidi nello spazio originale per visualizzarli
centroidi_originali = scaler.inverse_transform(kmeans.cluster_centers_)
axes[1].scatter(
    centroidi_originali[:, 0], 
    centroidi_originali[:, 1], 
    c='red', marker='X', s=300, edgecolors='black', linewidth=2,
    label='Centroidi'
)
axes[1].set_title(f'K-Means (K=3) | Silhouette={sil:.3f}')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("OSSERVAZIONI")
print("="*60)
print("""
✅ K-Means ha trovato esattamente i 3 gruppi
✅ I centroidi (stelle rosse) sono nel 'centro' di ogni cluster
✅ Silhouette alto (~0.7) indica buona separazione
✅ Con dati sferici e ben separati, K-Means funziona perfettamente
""")

---

## Demo 2: Visualizzare le iterazioni di K-Means

Mostriamo come l'algoritmo converge passo dopo passo.

In [ ]:
# ============================================
# DEMO 2: Visualizzare le iterazioni di K-Means
# ============================================
# Obiettivo: vedere come i centroidi si muovono ad ogni iterazione

np.random.seed(123)

# Dataset semplice per visualizzare bene
X_demo, _ = make_blobs(n_samples=150, centers=3, cluster_std=1.2, random_state=123)

# Inizializziamo i centroidi in modo random (NON k-means++)
# per vedere bene il movimento
np.random.seed(999)
initial_centroids = X_demo[np.random.choice(len(X_demo), 3, replace=False)]

# Eseguiamo K-Means manualmente per tracciare le iterazioni
def kmeans_step_by_step(X, K, initial_centroids, max_iter=10):
    """Esegue K-Means tracciando ogni iterazione"""
    centroids = initial_centroids.copy()
    history = [centroids.copy()]
    
    for iteration in range(max_iter):
        # Passo 1: Assegnazione - calcola distanze e assegna al centroide più vicino
        distances = np.zeros((len(X), K))
        for k in range(K):
            distances[:, k] = np.sqrt(np.sum((X - centroids[k])**2, axis=1))
        labels = np.argmin(distances, axis=1)
        
        # Passo 2: Aggiornamento - ricalcola i centroidi
        new_centroids = np.zeros_like(centroids)
        for k in range(K):
            if np.sum(labels == k) > 0:
                new_centroids[k] = X[labels == k].mean(axis=0)
            else:
                new_centroids[k] = centroids[k]
        
        # Controlla convergenza
        if np.allclose(centroids, new_centroids):
            break
        
        centroids = new_centroids
        history.append(centroids.copy())
    
    return labels, centroids, history

# Eseguiamo
labels_final, centroids_final, history = kmeans_step_by_step(X_demo, 3, initial_centroids)

print("="*60)
print("DEMO 2: Iterazioni di K-Means")
print("="*60)
print(f"\nCentroidi iniziali (random):")
for i, c in enumerate(initial_centroids):
    print(f"  Centroide {i}: ({c[0]:.2f}, {c[1]:.2f})")

print(f"\nNumero di iterazioni: {len(history)}")

# Visualizzazione delle iterazioni
n_plots = min(4, len(history))
fig, axes = plt.subplots(1, n_plots, figsize=(4*n_plots, 4))

iterations_to_show = [0, 1, len(history)//2, len(history)-1][:n_plots]

for idx, (ax, iter_num) in enumerate(zip(axes, iterations_to_show)):
    # Calcola le etichette per questa iterazione
    centroids_iter = history[iter_num]
    distances = np.zeros((len(X_demo), 3))
    for k in range(3):
        distances[:, k] = np.sqrt(np.sum((X_demo - centroids_iter[k])**2, axis=1))
    labels_iter = np.argmin(distances, axis=1)
    
    # Plot
    ax.scatter(X_demo[:, 0], X_demo[:, 1], c=labels_iter, cmap='viridis', s=40, alpha=0.6)
    ax.scatter(centroids_iter[:, 0], centroids_iter[:, 1], 
               c='red', marker='X', s=200, edgecolors='black', linewidth=2)
    
    if iter_num == 0:
        ax.set_title(f'Iterazione {iter_num}\n(centroidi random)')
    elif iter_num == len(history)-1:
        ax.set_title(f'Iterazione {iter_num}\n(CONVERGENZA)')
    else:
        ax.set_title(f'Iterazione {iter_num}')
    
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Mostra il movimento dei centroidi
print("\n" + "="*60)
print("MOVIMENTO DEI CENTROIDI")
print("="*60)
for i in range(3):
    start = initial_centroids[i]
    end = centroids_final[i]
    dist = np.sqrt(np.sum((end - start)**2))
    print(f"\nCentroide {i}:")
    print(f"  Partenza: ({start[0]:.2f}, {start[1]:.2f})")
    print(f"  Arrivo:   ({end[0]:.2f}, {end[1]:.2f})")
    print(f"  Distanza percorsa: {dist:.2f}")

print("\n💡 I centroidi si spostano verso il 'centro di massa' dei punti assegnati")

---

## Demo 3: Quando K-Means fallisce — cluster non sferici

Mostriamo cosa succede quando i cluster hanno forme che violano le assunzioni di K-Means.

In [ ]:
# ============================================
# DEMO 3: Quando K-Means fallisce
# ============================================
# Obiettivo: mostrare i limiti di K-Means su dati non ideali

from sklearn.datasets import make_moons, make_circles

np.random.seed(42)

# Creiamo 3 dataset problematici per K-Means
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# ============================================
# CASO 1: Cluster a forma di luna (moons)
# ============================================
X_moons, y_moons = make_moons(n_samples=300, noise=0.08, random_state=42)

# K-Means su moons
kmeans_moons = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_moons = kmeans_moons.fit_predict(X_moons)

# Plot
axes[0, 0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='coolwarm', s=40, alpha=0.7)
axes[0, 0].set_title('Moons: Gruppi VERI')
axes[0, 0].grid(True, alpha=0.3)

axes[1, 0].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_moons, cmap='coolwarm', s=40, alpha=0.7)
axes[1, 0].scatter(kmeans_moons.cluster_centers_[:, 0], kmeans_moons.cluster_centers_[:, 1],
                   c='black', marker='X', s=200, edgecolors='white', linewidth=2)
axes[1, 0].set_title('Moons: K-Means FALLISCE')
axes[1, 0].grid(True, alpha=0.3)

# ============================================
# CASO 2: Cerchi concentrici
# ============================================
X_circles, y_circles = make_circles(n_samples=300, noise=0.05, factor=0.5, random_state=42)

kmeans_circles = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_circles = kmeans_circles.fit_predict(X_circles)

axes[0, 1].scatter(X_circles[:, 0], X_circles[:, 1], c=y_circles, cmap='coolwarm', s=40, alpha=0.7)
axes[0, 1].set_title('Cerchi: Gruppi VERI')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 1].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_circles, cmap='coolwarm', s=40, alpha=0.7)
axes[1, 1].scatter(kmeans_circles.cluster_centers_[:, 0], kmeans_circles.cluster_centers_[:, 1],
                   c='black', marker='X', s=200, edgecolors='white', linewidth=2)
axes[1, 1].set_title('Cerchi: K-Means FALLISCE')
axes[1, 1].grid(True, alpha=0.3)

# ============================================
# CASO 3: Cluster di dimensioni molto diverse
# ============================================
# Un cluster grande, uno piccolo
X_big = np.random.randn(280, 2) * 2 + np.array([0, 0])
X_small = np.random.randn(20, 2) * 0.3 + np.array([6, 0])
X_unbalanced = np.vstack([X_big, X_small])
y_unbalanced = np.array([0]*280 + [1]*20)

kmeans_unbalanced = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_unbalanced = kmeans_unbalanced.fit_predict(X_unbalanced)

axes[0, 2].scatter(X_unbalanced[:, 0], X_unbalanced[:, 1], c=y_unbalanced, cmap='coolwarm', s=40, alpha=0.7)
axes[0, 2].set_title('Sbilanciati: Gruppi VERI\n(280 vs 20 punti)')
axes[0, 2].grid(True, alpha=0.3)

axes[1, 2].scatter(X_unbalanced[:, 0], X_unbalanced[:, 1], c=labels_unbalanced, cmap='coolwarm', s=40, alpha=0.7)
axes[1, 2].scatter(kmeans_unbalanced.cluster_centers_[:, 0], kmeans_unbalanced.cluster_centers_[:, 1],
                   c='black', marker='X', s=200, edgecolors='white', linewidth=2)
# Conta errori
errori = min((labels_unbalanced != y_unbalanced).sum(), 
             (labels_unbalanced != (1-y_unbalanced)).sum())
axes[1, 2].set_title(f'Sbilanciati: K-Means\n(errori: {errori})')
axes[1, 2].grid(True, alpha=0.3)

# Etichette righe
axes[0, 0].set_ylabel('GROUND TRUTH', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('K-MEANS', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# ============================================
# SPIEGAZIONE
# ============================================
print("="*70)
print("PERCHÉ K-MEANS FALLISCE IN QUESTI CASI?")
print("="*70)

print("""
📌 CASO 1 - MOONS (lune):
   I cluster hanno forma a "banana", non sferica.
   K-Means taglia verticalmente invece di seguire la curva.
   → Alternativa: DBSCAN (Lezione 23)

📌 CASO 2 - CERCHI CONCENTRICI:
   Un cluster è dentro l'altro.
   La distanza euclidea non può separare cerchi concentrici.
   → Alternativa: Spectral Clustering o DBSCAN

📌 CASO 3 - CLUSTER SBILANCIATI:
   Un cluster ha 280 punti, l'altro 20.
   K-Means tende a bilanciare, "rubando" punti dal cluster grande.
   → Alternativa: DBSCAN o algoritmi density-based
""")

---

## Demo 4: L'effetto degli outlier sui centroidi

Mostriamo come un singolo outlier può distorcere significativamente il risultato.

In [ ]:
# ============================================
# DEMO 4: L'effetto degli outlier
# ============================================
# Obiettivo: mostrare come gli outlier distorcono i centroidi

np.random.seed(42)

# Creiamo 2 cluster puliti
X_clean = np.vstack([
    np.random.randn(100, 2) * 0.5 + np.array([-2, 0]),
    np.random.randn(100, 2) * 0.5 + np.array([2, 0])
])
y_clean = np.array([0]*100 + [1]*100)

# Aggiungiamo alcuni outlier
outliers = np.array([
    [0, 8],     # outlier lontano in alto
    [-1, 7],    # altro outlier
    [1, 7.5]    # terzo outlier
])

X_with_outliers = np.vstack([X_clean, outliers])
y_with_outliers = np.array([0]*100 + [1]*100 + [2]*3)  # gli outlier come gruppo 2

# K-Means su dati puliti
kmeans_clean = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_clean = kmeans_clean.fit_predict(X_clean)

# K-Means su dati con outlier
kmeans_outliers = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_outliers = kmeans_outliers.fit_predict(X_with_outliers)

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Senza outlier
axes[0].scatter(X_clean[:, 0], X_clean[:, 1], c=labels_clean, cmap='coolwarm', s=50, alpha=0.7)
axes[0].scatter(kmeans_clean.cluster_centers_[:, 0], kmeans_clean.cluster_centers_[:, 1],
                c='black', marker='X', s=300, edgecolors='white', linewidth=2, label='Centroidi')
axes[0].set_title('SENZA outlier\n(centroidi corretti)')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-3, 10)

# Plot 2: Con outlier
# Coloriamo gli outlier in modo diverso
colors_outliers = ['blue' if l == 0 else 'red' for l in labels_outliers[:-3]]
axes[1].scatter(X_clean[:, 0], X_clean[:, 1], c=colors_outliers, s=50, alpha=0.7)
axes[1].scatter(outliers[:, 0], outliers[:, 1], c='green', s=150, marker='*', 
                edgecolors='black', linewidth=1, label='Outlier')
axes[1].scatter(kmeans_outliers.cluster_centers_[:, 0], kmeans_outliers.cluster_centers_[:, 1],
                c='black', marker='X', s=300, edgecolors='white', linewidth=2, label='Centroidi')
axes[1].set_title('CON outlier\n(centroidi DISTORTI)')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(-3, 10)

plt.tight_layout()
plt.show()

# Analisi numerica
print("="*60)
print("CONFRONTO CENTROIDI")
print("="*60)

print("\nCentroidi SENZA outlier:")
for i, c in enumerate(kmeans_clean.cluster_centers_):
    print(f"  Cluster {i}: ({c[0]:.2f}, {c[1]:.2f})")

print("\nCentroidi CON outlier:")
for i, c in enumerate(kmeans_outliers.cluster_centers_):
    print(f"  Cluster {i}: ({c[0]:.2f}, {c[1]:.2f})")

# Calcola lo spostamento
print("\nSpostamento dei centroidi:")
for i in range(2):
    spostamento = np.sqrt(np.sum((kmeans_clean.cluster_centers_[i] - 
                                   kmeans_outliers.cluster_centers_[i])**2))
    print(f"  Cluster {i}: spostamento = {spostamento:.2f}")

print("""
⚠️ ATTENZIONE:
Gli outlier hanno "tirato" uno dei centroidi verso l'alto.
Questo può causare assegnazioni sbagliate ai confini dei cluster.

💡 SOLUZIONI:
1. Rimuovere gli outlier prima del clustering
2. Usare algoritmi robusti come DBSCAN (identifica outlier come rumore)
3. Usare K-Medians invece di K-Means (meno sensibile)
""")

---

# Parte 4 — Esercizi Svolti

---

## Esercizio 20.1 — Calcolo manuale della distanza euclidea e del centroide

**Consegna:**
Dati i seguenti punti in 2D appartenenti a un cluster:
- A = (1, 2)
- B = (3, 4)
- C = (2, 1)
- D = (4, 3)

1. Calcola il centroide del cluster
2. Calcola la distanza euclidea di ogni punto dal centroide
3. Calcola l'inertia (somma delle distanze al quadrato)

In [ ]:
# ============================================
# ESERCIZIO 20.1 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 20.1 — Calcolo manuale")
print("="*70)

# Definiamo i punti
punti = np.array([
    [1, 2],  # A
    [3, 4],  # B
    [2, 1],  # C
    [4, 3]   # D
])
nomi = ['A', 'B', 'C', 'D']

print("\nPunti del cluster:")
for nome, punto in zip(nomi, punti):
    print(f"  {nome} = ({punto[0]}, {punto[1]})")

# ============================================
# PASSO 1: Calcolo del centroide
# ============================================
print("\n" + "="*70)
print("PASSO 1: Calcolo del centroide")
print("="*70)

# Il centroide è la media delle coordinate
centroide_x = np.mean(punti[:, 0])
centroide_y = np.mean(punti[:, 1])
centroide = np.array([centroide_x, centroide_y])

print(f"""
Formula: μ = (1/n) * Σ x_i

Calcolo x:
  μ_x = (1 + 3 + 2 + 4) / 4 = 10 / 4 = {centroide_x}

Calcolo y:
  μ_y = (2 + 4 + 1 + 3) / 4 = 10 / 4 = {centroide_y}

✅ Centroide: ({centroide_x}, {centroide_y})
""")

# ============================================
# PASSO 2: Calcolo delle distanze
# ============================================
print("="*70)
print("PASSO 2: Distanza di ogni punto dal centroide")
print("="*70)

print("\nFormula: d(P, μ) = √[(x_p - μ_x)² + (y_p - μ_y)²]\n")

distanze = []
for nome, punto in zip(nomi, punti):
    dx = punto[0] - centroide[0]
    dy = punto[1] - centroide[1]
    d = np.sqrt(dx**2 + dy**2)
    distanze.append(d)
    
    print(f"Punto {nome} = ({punto[0]}, {punto[1]}):")
    print(f"  dx = {punto[0]} - {centroide[0]} = {dx}")
    print(f"  dy = {punto[1]} - {centroide[1]} = {dy}")
    print(f"  d = √[({dx})² + ({dy})²] = √[{dx**2} + {dy**2}] = √{dx**2 + dy**2} = {d:.3f}")
    print()

# ============================================
# PASSO 3: Calcolo dell'inertia
# ============================================
print("="*70)
print("PASSO 3: Calcolo dell'Inertia (WCSS)")
print("="*70)

print("\nFormula: Inertia = Σ ||x_i - μ||²\n")

distanze_quadrato = [d**2 for d in distanze]
inertia = sum(distanze_quadrato)

print("Distanze al quadrato:")
for nome, d, d2 in zip(nomi, distanze, distanze_quadrato):
    print(f"  {nome}: {d:.3f}² = {d2:.3f}")

print(f"\nInertia = {' + '.join([f'{d2:.3f}' for d2 in distanze_quadrato])}")
print(f"        = {inertia:.3f}")
print(f"\n✅ Inertia del cluster: {inertia:.3f}")

# Verifica con numpy
print("\n" + "="*70)
print("VERIFICA CON NUMPY")
print("="*70)
inertia_numpy = np.sum((punti - centroide)**2)
print(f"Inertia calcolata con numpy: {inertia_numpy:.3f}")
print("✅ I calcoli sono corretti!" if np.isclose(inertia, inertia_numpy) else "❌ Errore nei calcoli")

# Visualizzazione
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(punti[:, 0], punti[:, 1], c='blue', s=100, label='Punti')
ax.scatter(centroide[0], centroide[1], c='red', marker='X', s=200, label='Centroide')

# Linee dal centroide ai punti
for nome, punto, d in zip(nomi, punti, distanze):
    ax.plot([centroide[0], punto[0]], [centroide[1], punto[1]], 'k--', alpha=0.5)
    ax.annotate(f'{nome}\nd={d:.2f}', (punto[0], punto[1]), textcoords="offset points", 
                xytext=(10, 5), fontsize=10)

ax.set_title(f'Cluster con centroide\nInertia = {inertia:.3f}')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
plt.show()

---

## Esercizio 20.2 — Clustering di clienti e-commerce

**Consegna:**
Un e-commerce ha i seguenti dati sui clienti:
- `frequenza`: numero di acquisti negli ultimi 12 mesi
- `spesa_media`: valore medio per acquisto (€)
- `recency`: giorni dall'ultimo acquisto

Applica K-Means per segmentare i clienti in 3 gruppi. Interpreta i cluster trovati.

In [ ]:
# ============================================
# ESERCIZIO 20.2 — SOLUZIONE
# ============================================

import pandas as pd

print("="*70)
print("ESERCIZIO 20.2 — Segmentazione clienti e-commerce")
print("="*70)

# ============================================
# PASSO 1: Generazione dataset realistico
# ============================================
np.random.seed(42)
n_clienti = 300

# Creiamo 3 profili di clienti (che l'algoritmo dovrà scoprire)
# Profilo 1: Clienti PREMIUM - frequenti, alta spesa, recenti
premium = pd.DataFrame({
    'frequenza': np.random.normal(25, 5, 80).clip(10, 40),
    'spesa_media': np.random.normal(150, 30, 80).clip(80, 250),
    'recency': np.random.normal(10, 5, 80).clip(1, 30)
})

# Profilo 2: Clienti OCCASIONALI - rari, spesa media, recenti
occasionali = pd.DataFrame({
    'frequenza': np.random.normal(5, 2, 120).clip(1, 12),
    'spesa_media': np.random.normal(50, 15, 120).clip(20, 100),
    'recency': np.random.normal(45, 15, 120).clip(15, 90)
})

# Profilo 3: Clienti DORMIENTI - erano attivi, ora inattivi
dormienti = pd.DataFrame({
    'frequenza': np.random.normal(12, 4, 100).clip(3, 25),
    'spesa_media': np.random.normal(80, 25, 100).clip(30, 150),
    'recency': np.random.normal(150, 40, 100).clip(90, 250)
})

# Combiniamo
df_clienti = pd.concat([premium, occasionali, dormienti], ignore_index=True)

print("\n📊 Dataset generato:")
print(df_clienti.describe().round(1))

# ============================================
# PASSO 2: Preprocessing - SCALING
# ============================================
print("\n" + "="*70)
print("PASSO 2: Scaling delle feature")
print("="*70)

print("\nRange PRIMA dello scaling:")
for col in df_clienti.columns:
    print(f"  {col}: {df_clienti[col].min():.1f} - {df_clienti[col].max():.1f}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clienti)

print("\nDopo StandardScaler: media≈0, std≈1 per tutte le feature")
print("✅ Le feature hanno ora lo stesso peso nelle distanze")

# ============================================
# PASSO 3: Applicazione K-Means
# ============================================
print("\n" + "="*70)
print("PASSO 3: K-Means con K=3")
print("="*70)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_scaled)

df_clienti['cluster'] = labels

print(f"\nRisultato:")
print(f"  Punti per cluster: {np.bincount(labels)}")
print(f"  Inertia: {kmeans.inertia_:.2f}")
print(f"  Silhouette: {silhouette_score(X_scaled, labels):.3f}")

# ============================================
# PASSO 4: Interpretazione dei cluster
# ============================================
print("\n" + "="*70)
print("PASSO 4: Interpretazione dei cluster")
print("="*70)

# Statistiche per cluster
cluster_stats = df_clienti.groupby('cluster').agg({
    'frequenza': ['mean', 'std'],
    'spesa_media': ['mean', 'std'],
    'recency': ['mean', 'std']
}).round(1)

print("\nStatistiche per cluster:")
print(cluster_stats)

# Interpretazione automatica basata sulle caratteristiche
print("\n" + "="*70)
print("INTERPRETAZIONE")
print("="*70)

for cluster_id in sorted(df_clienti['cluster'].unique()):
    cluster_data = df_clienti[df_clienti['cluster'] == cluster_id]
    freq_media = cluster_data['frequenza'].mean()
    spesa_media = cluster_data['spesa_media'].mean()
    recency_media = cluster_data['recency'].mean()
    n_clienti_cluster = len(cluster_data)
    
    print(f"\n📌 CLUSTER {cluster_id} ({n_clienti_cluster} clienti):")
    print(f"   Frequenza media: {freq_media:.1f} acquisti/anno")
    print(f"   Spesa media: €{spesa_media:.1f}")
    print(f"   Recency media: {recency_media:.1f} giorni")
    
    # Interpretazione
    if freq_media > 20 and recency_media < 30:
        print("   → PROFILO: 🌟 CLIENTI PREMIUM (fedeli, alto valore, attivi)")
        print("   → AZIONE: Programma loyalty, accesso anticipato, offerte esclusive")
    elif recency_media > 100:
        print("   → PROFILO: 😴 CLIENTI DORMIENTI (erano attivi, ora inattivi)")
        print("   → AZIONE: Campagna win-back, email personalizzata, sconto rientro")
    else:
        print("   → PROFILO: 🔄 CLIENTI OCCASIONALI (acquisti sporadici)")
        print("   → AZIONE: Aumentare engagement, cross-sell, newsletter")

# ============================================
# PASSO 5: Visualizzazione
# ============================================
print("\n" + "="*70)
print("PASSO 5: Visualizzazione")
print("="*70)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Frequenza vs Spesa
scatter1 = axes[0].scatter(df_clienti['frequenza'], df_clienti['spesa_media'], 
                            c=labels, cmap='viridis', s=50, alpha=0.7)
axes[0].set_xlabel('Frequenza (acquisti/anno)')
axes[0].set_ylabel('Spesa media (€)')
axes[0].set_title('Frequenza vs Spesa')
axes[0].grid(True, alpha=0.3)

# Plot 2: Frequenza vs Recency
scatter2 = axes[1].scatter(df_clienti['frequenza'], df_clienti['recency'], 
                            c=labels, cmap='viridis', s=50, alpha=0.7)
axes[1].set_xlabel('Frequenza (acquisti/anno)')
axes[1].set_ylabel('Recency (giorni)')
axes[1].set_title('Frequenza vs Recency')
axes[1].grid(True, alpha=0.3)

# Plot 3: Spesa vs Recency
scatter3 = axes[2].scatter(df_clienti['spesa_media'], df_clienti['recency'], 
                            c=labels, cmap='viridis', s=50, alpha=0.7)
axes[2].set_xlabel('Spesa media (€)')
axes[2].set_ylabel('Recency (giorni)')
axes[2].set_title('Spesa vs Recency')
axes[2].grid(True, alpha=0.3)

plt.colorbar(scatter3, ax=axes, label='Cluster', shrink=0.8)
plt.tight_layout()
plt.show()

print("\n✅ La segmentazione ha identificato 3 profili distinti di clienti")

---

### 📝 Esercizio 20.3 — Confronto: Con e senza Scaling

**Consegna:**
Hai un dataset con feature su scale molto diverse. Dimostra visivamente e quantitativamente la differenza tra applicare K-Means con e senza StandardScaler.

**Cosa deve emergere:**
1. Quanto cambiano le assegnazioni ai cluster
2. Quanto cambia la silhouette score
3. Perché lo scaling è cruciale

**Dataset:**
```python
feature_1 = [2, 3, 8, 9, 100]     # Scala: 2-100
feature_2 = [0.01, 0.02, 0.08, 0.09, 0.5]  # Scala: 0.01-0.5
```

**Hint:** Senza scaling, quale feature "domina" la distanza euclidea?

In [ ]:
# ============================================
# ESERCIZIO 20.3 — SOLUZIONE
# ============================================

print("="*70)
print("ESERCIZIO 20.3 — Effetto dello scaling su K-Means")
print("="*70)

# Dataset con scale diverse
np.random.seed(42)

# Generiamo dati con 3 cluster "reali" ma scale molto diverse
# Cluster 1
c1_f1 = np.random.normal(10, 2, 30)    # Scala grande
c1_f2 = np.random.normal(0.1, 0.02, 30) # Scala piccola

# Cluster 2
c2_f1 = np.random.normal(50, 5, 30)
c2_f2 = np.random.normal(0.5, 0.05, 30)

# Cluster 3
c3_f1 = np.random.normal(90, 3, 30)
c3_f2 = np.random.normal(0.9, 0.03, 30)

feature_1 = np.concatenate([c1_f1, c2_f1, c3_f1])
feature_2 = np.concatenate([c1_f2, c2_f2, c3_f2])
true_labels = np.array([0]*30 + [1]*30 + [2]*30)

X = np.column_stack([feature_1, feature_2])

print("\n📊 Range delle feature:")
print(f"  Feature 1: {feature_1.min():.1f} - {feature_1.max():.1f} (range: {feature_1.max()-feature_1.min():.1f})")
print(f"  Feature 2: {feature_2.min():.3f} - {feature_2.max():.3f} (range: {feature_2.max()-feature_2.min():.3f})")
print(f"\n⚠️  Feature 1 ha un range ~100x più grande di Feature 2!")

# ============================================
# SENZA SCALING
# ============================================
print("\n" + "="*70)
print("TEST A: K-Means SENZA Scaling")
print("="*70)

kmeans_no_scale = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_no_scale = kmeans_no_scale.fit_predict(X)

silhouette_no_scale = silhouette_score(X, labels_no_scale)

print(f"\nRisultato SENZA scaling:")
print(f"  Distribuzione: {np.bincount(labels_no_scale)}")
print(f"  Silhouette: {silhouette_no_scale:.3f}")

# ============================================
# CON SCALING
# ============================================
print("\n" + "="*70)
print("TEST B: K-Means CON StandardScaler")
print("="*70)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans_scaled = KMeans(n_clusters=3, random_state=42, n_init=10)
labels_scaled = kmeans_scaled.fit_predict(X_scaled)

silhouette_scaled = silhouette_score(X_scaled, labels_scaled)

print(f"\nRisultato CON scaling:")
print(f"  Distribuzione: {np.bincount(labels_scaled)}")
print(f"  Silhouette: {silhouette_scaled:.3f}")

# ============================================
# CONFRONTO
# ============================================
print("\n" + "="*70)
print("CONFRONTO")
print("="*70)

# Quanti punti cambiano cluster?
n_changed = np.sum(labels_no_scale != labels_scaled)
print(f"\n🔄 Punti che cambiano cluster: {n_changed}/{len(X)} ({100*n_changed/len(X):.1f}%)")
print(f"\n📈 Silhouette SENZA scaling: {silhouette_no_scale:.3f}")
print(f"📈 Silhouette CON scaling:   {silhouette_scaled:.3f}")
print(f"📈 Miglioramento:            +{100*(silhouette_scaled-silhouette_no_scale)/silhouette_no_scale:.1f}%")

# ============================================
# VISUALIZZAZIONE
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Dati originali con cluster veri
axes[0].scatter(X[:, 0], X[:, 1], c=true_labels, cmap='viridis', s=60, alpha=0.7)
axes[0].set_xlabel('Feature 1 (scala grande)')
axes[0].set_ylabel('Feature 2 (scala piccola)')
axes[0].set_title('Cluster VERI (ground truth)')
axes[0].grid(True, alpha=0.3)

# Plot 2: SENZA scaling
axes[1].scatter(X[:, 0], X[:, 1], c=labels_no_scale, cmap='viridis', s=60, alpha=0.7)
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].set_title(f'SENZA Scaling\nSilhouette: {silhouette_no_scale:.3f}')
axes[1].grid(True, alpha=0.3)

# Plot 3: CON scaling (visualizziamo i dati originali con le label scalate)
axes[2].scatter(X[:, 0], X[:, 1], c=labels_scaled, cmap='viridis', s=60, alpha=0.7)
axes[2].set_xlabel('Feature 1')
axes[2].set_ylabel('Feature 2')
axes[2].set_title(f'CON Scaling\nSilhouette: {silhouette_scaled:.3f}')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================
# LEZIONE CHIAVE
# ============================================
print("\n" + "="*70)
print("💡 LEZIONE CHIAVE")
print("="*70)
print("""
SENZA SCALING:
  • Feature 1 (range ~80) DOMINA completamente la distanza euclidea
  • Feature 2 (range ~0.8) viene praticamente IGNORATA
  • K-Means crea cluster basati quasi solo su Feature 1
  
CON SCALING:
  • Entrambe le feature contribuiscono equamente
  • I cluster riflettono la vera struttura dei dati
  • La silhouette score migliora
  
REGOLA: StandardScaler PRIMA di K-Means è quasi sempre necessario!
  Eccezione: quando le scale diverse sono significative (es. metri vs km)
""")

---

## 5. Cosa Portarsi a Casa

### ✅ Concetti Fondamentali

| Concetto | Definizione |
|----------|-------------|
| **K-Means** | Algoritmo che partiziona i dati in K cluster minimizzando l'inertia |
| **Centroide** | Punto medio di un cluster, calcolato come media delle coordinate |
| **Inertia (WCSS)** | Somma delle distanze quadrate dai punti ai centroidi |
| **k-means++** | Inizializzazione intelligente che distribuisce i centroidi iniziali |

### ⚠️ Errori Comuni da Evitare

| Errore | Problema | Soluzione |
|--------|----------|-----------|
| Non scalare i dati | Le feature con range grande dominano | Sempre `StandardScaler` prima |
| Scegliere K a caso | Cluster non significativi | Usare Elbow + Silhouette (Lezione 21) |
| Ignorare la forma dei cluster | K-Means fallisce con forme non convesse | Usare DBSCAN per forme arbitrarie (Lezione 23) |
| Una sola inizializzazione | Risultato dipende dal caso | Usare `n_init=10` o superiore |
| Non interpretare i cluster | Clustering inutile senza significato | Sempre analizzare le caratteristiche |

### 🔗 Ponte verso la Lezione 21

**Problema aperto:** Come scegliere K in modo sistematico?

In questa lezione abbiamo sempre "saputo" K a priori. Ma nella realtà:
- Non sappiamo quanti cluster ci siano
- Diverse scelte di K danno risultati diversi
- Servono metriche oggettive per decidere

**Nella prossima lezione:**
- Metodo del Gomito (Elbow Method)
- Silhouette Analysis approfondita
- Gap Statistic
- Trade-off interpretabilità vs performance

---

## 📋 BIGNAMI — Lezione 20: K-Means Clustering

### Definizioni Essenziali

| Termine | Definizione |
|---------|-------------|
| **K-Means** | Algoritmo di clustering che partiziona N punti in K cluster, assegnando ogni punto al cluster con centroide più vicino |
| **Centroide** | Centro geometrico di un cluster: $\mathbf{c}_k = \frac{1}{|C_k|} \sum_{i \in C_k} \mathbf{x}_i$ |
| **Inertia** | Within-Cluster Sum of Squares: $\text{WCSS} = \sum_{k=1}^{K} \sum_{i \in C_k} \|\mathbf{x}_i - \mathbf{c}_k\|^2$ |
| **Silhouette** | Misura di qualità del clustering: $s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$ |
| **k-means++** | Inizializzazione che sceglie centroidi iniziali distanti tra loro per evitare minimi locali |

---

### Formule Chiave

**Distanza Euclidea:**
$$d(\mathbf{x}, \mathbf{c}) = \sqrt{\sum_{j=1}^{d} (x_j - c_j)^2}$$

**Centroide di un cluster:**
$$\mathbf{c}_k = \frac{1}{|C_k|} \sum_{i \in C_k} \mathbf{x}_i$$

**Inertia (WCSS):**
$$J = \sum_{k=1}^{K} \sum_{i \in C_k} \|\mathbf{x}_i - \mathbf{c}_k\|^2$$

---

### Checklist K-Means

```
□ Feature selezionate (solo numeriche rilevanti)
□ StandardScaler applicato
□ K scelto con criterio (Elbow/Silhouette - Lezione 21)
□ n_init >= 10 per evitare minimi locali
□ Cluster interpretati e nominati
□ Visualizzazione per validare i risultati
□ Outlier gestiti se necessario
```

---

### Template di Codice

```python
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# 1. Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. K-Means
kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_scaled)

# 3. Valutazione
print(f"Inertia: {kmeans.inertia_:.2f}")
print(f"Silhouette: {silhouette_score(X_scaled, labels):.3f}")

# 4. Interpretazione
df['cluster'] = labels
df.groupby('cluster').mean()  # Profilo dei cluster
```

---

### Quando K-Means Funziona vs Quando Fallisce

| ✅ Funziona Bene | ❌ Fallisce |
|-----------------|-------------|
| Cluster sferici/convessi | Forme arbitrarie (mezzalune, anelli) |
| Cluster di dimensioni simili | Cluster molto sbilanciati |
| Dati ben separati | Cluster sovrapposti |
| Pochi outlier | Molti outlier (spostano i centroidi) |

---

### Parametri Chiave di KMeans

| Parametro | Default | Significato |
|-----------|---------|-------------|
| `n_clusters` | 8 | Numero di cluster K |
| `init` | 'k-means++' | Metodo di inizializzazione |
| `n_init` | 10 | Numero di inizializzazioni diverse |
| `max_iter` | 300 | Iterazioni massime per convergenza |
| `random_state` | None | Seed per riproducibilità |

---

### Flusso Mentale

```
DATI → StandardScaler → KMeans(K) → Valutazione → Interpretazione
         ↓                 ↓           ↓              ↓
      Normalizza        Itera      Silhouette    Significato
      le scale       converge?     Inertia       business
```

---

*"K-Means trova K centroidi che minimizzano le distanze intra-cluster. 
 Funziona meglio con cluster sferici, dati scalati, e K scelto con criterio."*